### Import data

In [1]:
import pandas as pd
df = pd.read_csv('../KPM/data/KPM_test_data.csv')
print(df.columns)

Index(['topic', 'stance', 'arg_id', 'argument', 'key_point_id', 'key_point',
       '4_7_8_15', '0_1_2_3', '5_6_9_10', '11_12_13_14', '16_17_18_19',
       '20_21_22_23', '24_25_26_27', 'avg', 'gpt4', 'llama2', 'qwen'],
      dtype='object')


### Calculate confidence score with qwen

In [2]:
import random
from http import HTTPStatus
import dashscope
import re
import yaml

In [ ]:
with open("../conf/index.yaml") as f:
    credentials = yaml.safe_load(f)
qwen_api_key = credentials['environment_variables']['QWEN_DASHSCOPE_API_KEY']

In [3]:
def generating(topic, argument,kp):
    try:
        messages = [
            {
                'role': 'system',
                'content': f"""
            You need to evaluate to what extend the 2 sentences matches to each other based on the topic "{topic}" and return a confidence score between 0 and 1. 0 represents totally not match, 1 represents very match. Your return should be in this format:Confidence Score: number
            """
            },
            {
                'role': 'user',
                'content': "argument:"+argument+", kp:"+kp
            }
        ]
        dashscope.api_key = qwen_api_key
        response = dashscope.Generation.call(model="qwen-max",
                                   messages=messages,
                                   # 设置随机数种子seed，如果没有设置，则随机数种子默认为1234
                                   seed=random.randint(1, 10000),
                                   # 将输出设置为"message"格式
                                   result_format='message')
        if response.status_code == HTTPStatus.OK:
            output = response['output']["choices"][0]["message"]["content"]
            print(response['output']["choices"][0]["message"]["content"])
            # confidence_score_match = re.findall(r'Confidence Score: (\d+\.\d+)', output)
            confidence_score_match = re.findall(r'Confidence Score: (\d+(\.\d+)?)', output)

            if confidence_score_match:
                print(confidence_score_match)
                score = float(confidence_score_match[0][0])
                print("KP score: "+str(score))
                print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
                return score
            else:
                print(f"Error. Regenerating string...")
                return generating(topic, argument, kp)
        else:
            print('Request id: %s, Status code: %s, error code: %s, error message: %s' % (
                response.request_id, response.status_code,
                response.code, response.message
            ))
            return generating(topic, argument, kp)
    except Exception as e:
        print(f"An error occurred: {e}. Retrying...")
        return generating(topic, argument, kp)

In [24]:
import ast
for index,row in df.iloc[3500:].iterrows():
    topic = row['topic']
    kp = row['key_point']
    argument = row['argument']
    print(index)
    confidence_score =generating(topic,argument,kp)
    print("Confidence Score: "+ str(confidence_score))
    df.at[index,'qwen'] = confidence_score

3500
Confidence Score: 0.9
[('0.9', '.9')]
KP score: 0.9
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Confidence Score: 0.9
3501
Confidence Score: 0.8
[('0.8', '.8')]
KP score: 0.8
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Confidence Score: 0.8
3502
Confidence Score: 0.8
[('0.8', '.8')]
KP score: 0.8
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Confidence Score: 0.8
3503
Confidence Score: 0.9
[('0.9', '.9')]
KP score: 0.9
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Confidence Score: 0.9
3504
Confidence Score: 1
[('1', '')]
KP score: 1.0
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Confidence Score: 1.0
3505
Confidence Score: 0.75
[('0.75', '.75')]
KP score: 0.75
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Confidence Score: 0.75
3506
Confidence Score: 0.7
[('0.7', '.7')]
KP score: 0.7
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Confidence Score: 0.7
3507
Confidence Score: 0.9
[('0.9', '.9')]
KP score: 0.9
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Confidence Score: 0.9
3508
Confidence Score: 0.8
[('0.8

In [26]:
df.to_csv('../KPM/data/KPM_test_data_2.csv', index=False)
print("Add new data successfully!!!!!")

Add new data successfully!!!!!
